## Biblioteki

In [1]:
import requests
from bs4 import BeautifulSoup

In [9]:
product_id = "11740014"

url = f"https://www.ceneo.pl/114700014" #https://www.ceneo.pl/114700014
response = requests.get(url)
response.status_code

200

In [10]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion=opinions.pop(0)
print(type(opinion))
print(opinion)

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<div class="user-post user-post__card js_product-review" data-entry-id="17516357">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/8.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
g...a</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2023-05-17 10:52:54">11 miesięcy temu, </time>
<time datetime="2023-05-12 09:57:19">po 5 dniach</time> użytkowania

# Ekstrakcja składowych pojedynczych opinii
| Składowa | Selektor | Zmienna |
|----------|----------|---------|
| id opinii| ["data-entry-id"] | opinion_id|
| autor    | span.user-post__author-name |author |
| rekomendacja| span.user-post__author-recomendation > em |recommendation|
| gwiazdki | span.user-post__score-count| rating|
| treść    | div.user-post__text|content | 
| lista zalet |div.review-feature__title review-feature__title--positives | pros|
| lista wad | div.review-feature__title review-feature__title--negatives| cons|
| dla ilu przydatna | span[id^="vote-yes"]| useful|
| dla ilu nieprzydatna| span[id^="vote-no"]| useless|
| data wystawienia | span.user-post__published > time:nth-child(1)["datetime"]| post_date|
|data zakupu | span.user-post__published > time:nth-child(2)["datetime"] | purchase_rate |

In [ ]:
opinion_id = opinion[['data-entry-id']]
author = opinion.select_one("span.user-post__author-name")
recommendation = opinion.select_one('span.user-post__author-recomendation > em')
rating = opinion.select_one("span.user-post__score-count")
content = opinion.select_one("div.user-post__text")
pros = opinion.select_one("div.review-feature__title review-feature__title--positives")
cons = opinion.select_one("div.review-feature__title review-feature__title--negatives")
useful = opinion.select_one('span[id^="vote-yes"]')
useless = opinion.select_one('span[id^="vote-no"]')
post_date = opinion.select_one('span.user-post__published > time:nth-child(1)["datetime"]')
purchase_date  = opinion.select_one('span.user-post__published > time:nth-child(2)["datetime"]')
